In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf

from tensorflow.keras import layers

from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

from vit_for_tflite_compartible import create_vit_classifier, PatchEncoder

import numpy as np
# from tensorflow_model_optimization.python.core.keras.compat import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
    extract_patches_from_audio_dataset
)
from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    )


2024-07-28 23:24:57.980639: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 23:24:58.337563: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 23:24:58.337659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 23:24:58.405245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 23:24:58.572202: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 23:24:58.573164: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(train_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-07-28 23:25:04.613282: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-28 23:25:04.613945: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2024-07-28 23:25:04.798213: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-28 23:25:04.854938: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [4]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)

# x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
# x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
# x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [5]:
train_patches_ds = extract_patches_from_audio_dataset(train_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)
test_patches_ds = extract_patches_from_audio_dataset(test_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)
val_patches_ds = extract_patches_from_audio_dataset(val_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_patches_ds, data_type="not spectrogram")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_patches_ds, data_type="not spectrogram")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_patches_ds, data_type="not spectrogram")

2024-07-28 23:25:07.968243: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24117248 exceeds 10% of free system memory.
2024-07-28 23:25:07.988644: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24164352 exceeds 10% of free system memory.
2024-07-28 23:25:08.053297: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24117248 exceeds 10% of free system memory.
2024-07-28 23:25:08.084929: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24164352 exceeds 10% of free system memory.
2024-07-28 23:25:08.173805: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24117248 exceeds 10% of free system memory.


In [7]:
for i in train_patches_ds:
    print(i)
    break

(<tf.Tensor: shape=(32, 23, 10, 64), dtype=float32, numpy=
array([[[[-8.56713200e+00, -8.11090660e+00, -7.47468662e+00, ...,
          -7.00050926e+00, -6.81739902e+00, -6.23985577e+00],
         [-6.23447227e+00, -5.54868650e+00, -5.06280613e+00, ...,
          -4.42214298e+00, -4.68557405e+00, -5.01444292e+00],
         [-3.28225708e+00, -3.11301279e+00, -2.77596331e+00, ...,
          -2.20940375e+00, -2.14740610e+00, -2.08745575e+00],
         ...,
         [ 5.40027142e-01,  6.38262510e-01,  8.25802445e-01, ...,
          -2.14161468e+00, -2.36601543e+00, -2.02531838e+00],
         [ 2.83198237e-01, -4.59740400e-01, -9.15585399e-01, ...,
          -1.20322204e+00, -8.60022664e-01, -5.28287530e-01],
         [-1.42150664e+00, -9.13994908e-01, -9.50380206e-01, ...,
          -1.69635975e+00, -3.07473469e+00, -5.06058025e+00]],

        [[-8.67402267e+00, -7.74950314e+00, -6.79012728e+00, ...,
          -5.32791185e+00, -5.19800472e+00, -5.32412863e+00],
         [-5.77756071e+00, -5

In [8]:
for i in x_val_np:
    print(i.shape)
    break

(23, 10, 64)


Attention mechnism is mechanism that allows each element of a sequence to focus on other elements of the same sequence.

queries (
𝑄
Q), keys (
𝐾
K), and values (
𝑉
V) understand this

In transformers, the MLP is part of the transformer block and is applied after the self-attention mechanism. It consists of two or more fully connected (dense) layers with a non-linear activation function applied between them. 

We give to model not image but patches

In [68]:
input_shape = (23, 10, 64)  
# input_shape = (46, 20, 16)  
num_classes = 2  
patch_size = 8
num_patches = (184 // patch_size) * (80 // patch_size) # 230
num_patches

230

In [117]:
input_shape = (23, 10, 64)
# input_shape = (46, 20, 16)  
num_classes = 2  
patch_size = 8
num_patches = (184 // patch_size) * (80 // patch_size) # 230

projection_dim = 8  # Size of embedding hpo # Less was tested - not successfull
num_heads = 4  # Number of attention heads hpo
transformer_layers = 1  # Number of transformer layers
mlp_head_units = [16, 8]  # Multi-Layer Perceptron head units
transformer_units = [projection_dim * 2, projection_dim]

vit_classifier = create_vit_classifier(input_shape, num_classes, patch_size, num_patches, projection_dim, num_heads, transformer_layers, transformer_units, mlp_head_units)
vit_classifier.summary()
vit_classifier.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_40 (InputLayer)       [(None, 23, 10, 64)]         0         []                            
                                                                                                  
 reshape_39 (Reshape)        (None, 230, 64)              0         ['input_40[0][0]']            
                                                                                                  
 dense_197 (Dense)           (None, 230, 8)               520       ['reshape_39[0][0]']          
                                                                                                  
 patch_encoder_36 (PatchEnc  (None, 230, 8)               1840      ['dense_197[0][0]']           
 oder)                                                                                     

In [118]:
vit_classifier.fit(train_patches_ds, epochs=20, validation_data=val_patches_ds)
print("Validation dataset accuracy:")
val_loss, val_acc = vit_classifier.evaluate(x_val_np, y_val_np)

Epoch 1/20
353/353 [==============================] - 35s 91ms/step - loss: 0.6135 - accuracy: 0.6676 - val_loss: 0.3027 - val_accuracy: 0.8993
Epoch 2/20
353/353 [==============================] - 32s 90ms/step - loss: 0.4136 - accuracy: 0.7673 - val_loss: 0.1892 - val_accuracy: 0.9326
Epoch 3/20
353/353 [==============================] - 32s 90ms/step - loss: 0.3249 - accuracy: 0.8402 - val_loss: 0.1535 - val_accuracy: 0.9486
Epoch 4/20
353/353 [==============================] - 31s 89ms/step - loss: 0.2817 - accuracy: 0.8683 - val_loss: 0.1366 - val_accuracy: 0.9493
Epoch 5/20
353/353 [==============================] - 31s 89ms/step - loss: 0.2515 - accuracy: 0.8843 - val_loss: 0.1371 - val_accuracy: 0.9609
Epoch 6/20
353/353 [==============================] - 32s 92ms/step - loss: 0.2154 - accuracy: 0.9031 - val_loss: 0.1044 - val_accuracy: 0.9616
Epoch 7/20
353/353 [==============================] - 31s 89ms/step - loss: 0.2087 - accuracy: 0.9131 - val_loss: 0.1095 - val_accuracy:

In [10]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(vit_classifier, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(vit_classifier, x_test_np, y_test_np, model_format="keras")

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(vit_classifier, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True)


Validation dataset:
44/44 [==============================] - 2s 42ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.68%
Recall: 97.03%
Precision: 96.22%
F1-score: 96.62%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.966087575828162

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9655628873374007

Test dataset:
44/44 [==============================] - 2s 40ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9685420863541188

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9683766834475156
Time of one prediction for Test dataset:
Accuracy: 97.85%
Recall: 96.51%
P

In [122]:
MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/vit/hpo/vit_mel_spec_final.keras"
vit_classifier.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "MB")

Model file name:  ../spectrogram_models_from_notebooks/vit/hpo/vit_mel_spec_final.keras
File size: 0.482 Megabytes


In [123]:
model_loaded = tf.keras.models.load_model(MODEL_FILE_NAME, compile=True, custom_objects={ "PatchEncoder": PatchEncoder})
